In [1]:
# import 
from keras.datasets import cifar100
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Input, Dense, Flatten, BatchNormalization, Activation, add
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical

import argparse
import numpy as np

Using TensorFlow backend.


In [0]:
# model作成 CNNByChainerと同じ層構成にする
def create_CNN_model(input_shape=(32, 32, 3), class_num=100):
    input = Input((32, 32, 3))
    kernel_size = (3, 3)
    max_pool_size = (3, 3)
    # 畳み込み層の実装
    # 1層目
    cnn = Conv2D(128, kernel_size=kernel_size, padding='same', strides=(1, 1), activation='relu', input_shape=(32, 32, 3))(input)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Conv2D(256, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Dropout(0.25)(cnn)
    cnn = Conv2D(512, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Conv2D(1024, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)
    cnn = Dropout(0.25)(cnn)
    cnn = Conv2D(1024, kernel_size, padding='same', strides=(1, 1), activation='relu')(cnn)

    # 入力を平滑化する層（いわゆるデータをフラット化する層、例えば4次元配列を1次元配列に変換するなど）
    fc = Flatten()(cnn)
    # denseは全結合層
    fc = Dense(2048, activation='relu')(fc)
    fc = Dropout(0.5)(fc)
    softmax = Dense(100, activation='softmax')(fc)
    model = Model(input=input, output=softmax)
    
    return model

def train():

    # cifar10のデータ取得
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    x_train1, x_valid, y_train1, y_valid = train_test_split(x_train, y_train, test_size=0.2)
    
    model = create_CNN_model()

    # RGB画像で32×32なので32×32×3にreshapeする
    x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
    x_valid = x_valid.reshape(x_valid.shape[0], 32, 32, 3)
    x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)
    
    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_valid /= 255
    x_test /= 255

    # one-hot vector形式に変換する
    y_train = to_categorical(y_train, 100)
    y_valid = to_categorical(y_valid, 100)
    y_test = to_categorical(y_test, 100)
    
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(),  metrics=['accuracy'])
    # 学習
    model.fit(x_train, y_train, epochs=40, batch_size=256, verbose=1, validation_data=(x_valid, y_valid))
    # 精度算出
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [23]:
train()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 42s 849us/step - loss: 4.3538 - acc: 0.0258 - val_loss: 4.0736 - val_acc: 0.0497
Epoch 2/40
50000/50000 [==============================] - 34s 684us/step - loss: 3.9126 - acc: 0.0771 - val_loss: 3.6424 - val_acc: 0.1152
Epoch 3/40
50000/50000 [==============================] - 34s 684us/step - loss: 3.5891 - acc: 0.1292 - val_loss: 3.3623 - val_acc: 0.1636
Epoch 4/40
50000/50000 [==============================] - 34s 687us/step - loss: 3.3136 - acc: 0.1769 - val_loss: 3.0600 - val_acc: 0.2232
Epoch 5/40
50000/50000 [==============================] - 35s 690us/step - loss: 3.0469 - acc: 0.2316 - val_loss: 2.7859 - val_acc: 0.2819
Epoch 6/40
50000/50000 [==============================] - 34s 689us/step - loss: 2.8127 - acc: 0.2761 - val_loss: 2.5436 - val_acc: 0.3300
Epoch 7/40
50000/50000 [==============================] - 34s 690us/step - loss: 2.6232 - acc: 0.3156 - val_loss: 2.